# Stabilizatión of the simulation

In [1]:
## -----Import essential packages

from __future__ import division
import hoomd
import hoomd.md

#-----Define important variables

temp = 9.0
pressure = 0.1
#-----Start from a gsd file

hoomd.context.initialize("--mode=cpu", memory_traceback = True);

#-----Select the trayectory of the center of mass of particles

system = hoomd.init.read_gsd("T_CM_9.0_0.1_ramp.gsd",frame = -1)

#-----Define each rigid body type in the local coordinate system of the body.

rigid = hoomd.md.constrain.rigid();
rigid.set_param('M', 
                types=['A']*8,
                positions=[(-4,0,0),(-3,0,0),(-2,0,0),(-1,0,0),
                           (1,0,0),(2,0,0),(3,0,0),(4,0,0)]);

#-----Instruct the rigid constraint to create the constituent particles

rigid.create_bodies();

#-----Define the potential energy

nl = hoomd.md.nlist.cell()

lj = hoomd.md.pair.lj(r_cut = 3.5, nlist=nl)
lj.set_params(mode='shift')

#-----Define interaction with species in the binary mixture

lj.pair_coeff.set('M','M', epsilon = 1.0, sigma = 1.0)
lj.pair_coeff.set('A','A', epsilon = 1.0, sigma = 1.0)
lj.pair_coeff.set('M','A', epsilon = 1.0, sigma = 1.0)

#-----Select an standard integrator

hoomd.md.integrate.mode_standard(dt=0.005);

#------Define group

mesogens = hoomd.group.rigid_center();

#----Integrate using NPT

npt = hoomd.md.integrate.npt(group = mesogens, kT = temp, tau = 2.5, tauP = 1.5, P = pressure);

#-----Write output and Run the Simulation

log_file = "T_" + str(temp) + "_" + str(pressure) +"_stabilization.log"
gsd_file = "T_" + str(temp) + "_" + str(pressure) +"_stabilization.gsd" 
meso_gsd_file = "T_CM_" + str(temp) + "_" + str(pressure) + "_stabilization.gsd"
log = hoomd.analyze.log(filename = log_file,
                         quantities = ['num_particles',
                                     'ndof',
                                     'translational_ndof',
                                     'rotational_ndof',
                                     'potential_energy',
                                     'kinetic_energy',
                                     'translational_kinetic_energy',
                                     'rotational_kinetic_energy',
                                     'temperature',
                                     'pressure',
                                      'volume'],
                         period = 1e3,
                         overwrite = True);
gsd = hoomd.dump.gsd(gsd_file,
               period = 1e3,
               group = hoomd.group.all(),
               overwrite=True); 
meso_gsd = hoomd.dump.gsd(meso_gsd_file,
               period = 1e3,
               group = mesogens,
               overwrite=True); 

HOOMD-blue 2.6.0 DOUBLE HPMC_MIXED TBB SSE SSE2 SSE3 
Compiled: 05/29/2019
Copyright (c) 2009-2019 The Regents of the University of Michigan.
-----
You are using HOOMD-blue. Please cite the following:
* J A Anderson, C D Lorenz, and A Travesset. "General purpose molecular dynamics
  simulations fully implemented on graphics processing units", Journal of
  Computational Physics 227 (2008) 5342--5359
* J Glaser, T D Nguyen, J A Anderson, P Liu, F Spiga, J A Millan, D C Morse, and
  S C Glotzer. "Strong scaling of general-purpose molecular dynamics simulations
  on GPUs", Computer Physics Communications 192 (2015) 97--107
-----
HOOMD-blue is running on the CPU
notice(2): Group "all" created containing 1000 particles
notice(2): constrain.rigid(): Creating 1000 rigid bodies (adding 8000 particles)
notice(2): Group "rigid_center" created containing 1000 particles


In [2]:
hoomd.run(5e5)

notice(2): -- Neighborlist exclusion statistics -- :
notice(2): Particles with 0 exclusions             : 9000
notice(2): Neighbors included by diameter          : no
notice(2): Neighbors excluded when in the same body: yes
** starting run **
Time 00:00:10 | Step 1826605 / 2326000 | TPS 60.4244 | ETA 02:17:44
Time 00:00:20 | Step 1827340 / 2326000 | TPS 73.4636 | ETA 01:53:07
Time 00:00:30 | Step 1828079 / 2326000 | TPS 73.8129 | ETA 01:52:25
Time 00:00:40 | Step 1828767 / 2326000 | TPS 68.6533 | ETA 02:00:42
Time 00:00:50 | Step 1829447 / 2326000 | TPS 67.9908 | ETA 02:01:43
Time 00:01:00 | Step 1830191 / 2326000 | TPS 74.2523 | ETA 01:51:17
Time 00:01:10 | Step 1830937 / 2326000 | TPS 74.4503 | ETA 01:50:49
Time 00:01:20 | Step 1831624 / 2326000 | TPS 68.695 | ETA 01:59:56
Time 00:01:30 | Step 1832307 / 2326000 | TPS 68.284 | ETA 02:00:29
Time 00:01:40 | Step 1833033 / 2326000 | TPS 72.4887 | ETA 01:53:20
Time 00:01:50 | Step 1833793 / 2326000 | TPS 75.9804 | ETA 01:47:58
Time 00:02:

**ERROR**: Particle with unique tag 101 is no longer in the simulation box.

**ERROR**: Cartesian coordinates: 
**ERROR**: x: 12.0944 y: 32.6555 z: 16.575
**ERROR**: Fractional coordinates: 
**ERROR**: f.x: 0.730225 f.y: 1.12162 f.z: 0.815515
**ERROR**: Local box lo: (-26.2666, -26.2666, -26.2666)
**ERROR**:           hi: (26.2666, 26.2666, 26.2666)


RuntimeError: Error computing cell list